# Coregistration of two Sentinel-2 scenes with a Landsat-8 reference scene <img align="right" src="../Supplementary_data/dea_logo.jpg">

* **[Sign up to the DEA Sandbox](https://app.sandbox.dea.ga.gov.au/)** to run this notebook interactively from a browser
* **Compatibility:** Notebook currently compatible with `DEA Sandbox` environments
* **Products used:** 
[landsat-c2l2-sr](https://landsatlook.usgs.gov/stac-server/collections/landsat-c2l2-sr)
[sentinel-2-l2a](https://earth-search.aws.element84.com/v1/collections/sentinel-2-l2a)


## Background
Satellite image co-registration ensures the accurate geometric alignment of multi-temporal or multi-sensor images. It is critical for applications like change detection, data fusion, and displacement mapping. Co-registration of satellite images is a critical process in remote sensing, enabling accurate multi-temporal and multi-sensoral analysis. This process aligns images taken at different times or by different sensors to a common coordinate system, reducing misalignments caused by geometric errors. For more details on co-registration and its available tools please see the references below:

[Satelite Image Co-Registration](https://geoscienceau.sharepoint.com/:b:/s/DEAnt/ERAjz6UbYLJGh20wvObq2YwBZ-QNrGk37Qz5ObjLQNJQfA?e=lKubfl)

[Co-Registration Tools Comparison](https://geoscienceau.sharepoint.com/:b:/s/DEAnt/Efz-tYil3pVDv3nrvMIYBHoBlZbvUwKTc1aOOpVYPPX7Ig?e=0hMtWp)

## Description
This notebooks tests a optical images cor-registeration of two Sentinel-2 target scenese with a Landsat-8 reference one, using three different tools and compares their results. `Co-register` , `Karios` and `AROSICS`. The first two are mainly based on Optical Flow tracking of the features in the scenes and the last one mainly uses Phase Correlation. `Co-register` is built in-house in GA and is compared with the other two. 

For more information about Karios ansd AROSICS, refer to the links below:

[Karios](https://github.com/telespazio-tim/karios)

[AROSICS](https://github.com/GFZ/arosics)

1. First we download and prepare the reference and target scenes.
2. Second, we run three different co-registration tools using the input scenes.
3. Finally we compare the results using the available metrics for image similarity.

>**Note:** 
>* All parameters are defualt parameters, except for AROSICS, where the min reliability was dropped to 30%.
>* Shifts are translation shifts only. Mean shifts (Average shifts in both x and y directions from all detected features) are used for final co-registration.
***

## Getting started

AROSICS and KArios should be installed in your environment. 

[AROSICS intallation](https://danschef.git-pages.gfz-potsdam.de/arosics/doc/installation.html)

For Karios, you can follow the link below:

[Karios installation](https://github.com/telespazio-tim/karios?tab=readme-ov-file#installation)

or you can just install from the main branch of the repo by running:


```pip install git+https://github.com/telespazio-tim/karios.git@main```



### Load packages
Import Python packages that are used for the analysis.

In [ ]:
%matplotlib inline

from pystac_client import Client
import boto3
import rasterio as rio
import matplotlib.pyplot as plt
import numpy as np
import cv2 as cv
import geopandas as gpd
from skimage.exposure import rescale_intensity
import os
import glob
import pandas as pd
from rasterio.plot import show
import planetary_computer

# import sys
# sys.path.insert(1, "../Tools/")
from utils import reproject_tif, stream_scene, co_register, arosics, karios

### Querying data using PyStac

We use landsat-8 with scene id `LC08_L2SR_128111_20200228_02_T2` as the reference image and Sentinel-2 scenes with ids `S2B_42CVE_20250124_0_L2A`, `S2B_42CVE_20250110_0_L2A`, as target or monitored scenes to be co-registered with the refernce scene. These scenes are all from the Amery ice shelf in Antarctica.

In [ ]:
ref_ids = ["LC08_L2SR_128111_20200228_02_T2"]
tgt_ids = ["S2B_42CVE_20250124_0_L2A", "S2B_42CVE_20250110_0_L2A"]

### Create input and outpur folders

In [ ]:
input_dirs = "temp_dea_coreg/inputs"
output_dirs = "temp_dea_coreg/outputs"
os.makedirs(input_dirs, exist_ok=True)
os.makedirs(output_dirs, exist_ok=True)

Co-registration can be run only on single bands or on multi-band images. For this test we use an averaged greyscale image of a RGB composite of 3 seperate bands `red`, `green` and `blue`, with contrast stretching as input image pre-enhancement step.

In [ ]:
bands = ["SR_B4", "SR_B3", "SR_B2"]  # specifying the bands to be used for RGB composite
# bands = ["red", "green", "blue"]  # specifying the bands to be used for RGB composite

### Querying and downloading the scenes

In [ ]:
# Planetary Computer assets are held in private Azure blobs. We need to sign the assets to access them. This is done via the right modifier passed to pystac client.
server_url = "https://planetarycomputer.microsoft.com/api/stac/v1"
query = {"ids": ref_ids, "collections": ["landsat-8-c2-l2"]}
client = Client.open(server_url, modifier=planetary_computer.sign_inplace)
search = client.search(**query)
ref_items = search.item_collection()
print(len(ref_items))

In [ ]:
# # querying the reference scene using PyStac
# server_url = "https://landsatlook.usgs.gov/stac-server"
# query = {"ids": ref_ids}
# client = Client.open(server_url, headers=headers)
# search = client.search(**query)
# ref_items = search.item_collection()
# display(ref_items)

Downloading the reference scene bands from the cloud using rasterio.

Landsat-8 data on AWS is stored in a requester-pays bucket, so we need to create a session that supports this if downloading from AWS. Planetary Computer scenes are hosted on Azure. The Urls for them need to be signed before we can download them. We have already signed the assets in place when opening the pystac client.

`stream_scene` function is defined in `tools.coregister` library , and it returns the data as a numpy array and its metadata. We can scale the data while streaming to reduce memory usage. Here we set the output resolution to 200.0 meters.

In [ ]:
bands_data = []  # stores data for each band
for band in bands:
    band_url = ref_items[0].assets[band].href
    data = stream_scene(band_url, resolution=200.0, round_transform=False)
    bands_data.append(data)

In [ ]:
# aws_session = rio.session.AWSSession(boto3.Session(), requester_pays=True)
# bands_data = []  # stores data for each band
# for band in bands:
#     band_url = ref_items[0].assets[band].to_dict()["alternate"]["s3"]["href"]
#     data = stream_scene(
#         band_url, aws_session=aws_session, resolution=200.0, round_transform=False
#     )
#     bands_data.append(data)

In [ ]:
# merging the bands into a single RGB image
bands_images = [np.nan_to_num(np.squeeze(datum[0])) for datum in bands_data]
composite_image = cv.merge(bands_images)

In [ ]:
# plotting the composite image for inspection
plt.imshow(composite_image / composite_image.max())

### Pre-processing
We now create a grayscale image and apply contrast enhancement on it to create a more informative reference image. 
Karios and AROSICS work on a single band only (even if the input is a multi-band image), therefore we use the code below to create a grey reference image by averaging the bands in the true colour image. Please note, this is not the usual grayscale image that image processing tools such as OpenCV generate. The normal conversion is according NTSC formula which preserves the relationship between bands to align better with human perception. 
Also we stretch the contrast of the composite images. This will reveal more information about the features in the images, potentially.

In [ ]:
# Creating a grayscale version of the reference image for use in registration
# using the 2nd and 98th percentiles to stretch the contrast for image enhancement
# and saving it to disk

grey_image = np.mean(composite_image, axis=2)
p2, p98 = np.percentile(grey_image, (0, 98))
ref_grey = rescale_intensity(grey_image, in_range=(p2, p98), out_range="uint8")
ref_profile = bands_data[0][1]["profile"]
ref_profile["dtype"] = "uint8"
with rio.open(f"{input_dirs}/ref_image.tif", "w", **ref_profile) as dst:
    dst.write(ref_grey, 1)

Dwonloading and processing the target scenes

In [ ]:
server_url = "https://earth-search.aws.element84.com/v1"
query = {"ids": tgt_ids}
client = Client.open(server_url)
search = client.search(**query)
tgt_items = search.item_collection()
print(len(tgt_items))

In [ ]:
bands = ["red", "green", "blue"]  # specifying the bands to be used for RGB composite
aws_session = rio.session.AWSSession(boto3.Session())
items_data = []
for item in tgt_items:
    print(item.id)
    bands_data = []
    for band in bands:
        band_url = item.assets[band].href
        data = stream_scene(
            band_url, aws_session=aws_session, resolution=200.0, round_transform=False
        )
        bands_data.append(data)
    items_data.append({"id": item.id, "bands": bands_data})

In [ ]:
tgt_composite_images = []
for idx, item_data in enumerate(items_data):
    bands_data = item_data["bands"]
    bands_images = [np.nan_to_num(np.squeeze(datum[0])) for datum in bands_data]
    composite_image = cv.merge(bands_images)
    tgt_composite_images.append(composite_image)

plt.imshow(tgt_composite_images[0] / tgt_composite_images[0].max())

Converting the targets to grayscale images and applying contrast enhancement.

In [ ]:
## grayscale target images

tgt_greys = []
for idx, composite_image in enumerate(tgt_composite_images):
    grey_image = np.mean(composite_image, axis=2)
    p2, p98 = np.percentile(grey_image, (0, 98))
    tgt_image = rescale_intensity(grey_image, in_range=(p2, p98), out_range="uint8")
    tgt_greys.append(tgt_image)
    tgt_profile = items_data[idx]["bands"][0][1]["profile"]
    tgt_profile["dtype"] = "uint8"
    with rio.open(f"{input_dirs}/tgt_image_{idx}.tif", "w", **tgt_profile) as dst:
        dst.write(tgt_image, 1)

Plotting the reference and target images next to each  other

In [ ]:
fig, axes = plt.subplots(1, 3, figsize=(12, 36), dpi=200)
show(
    rio.open(f"{input_dirs}/ref_image.tif"),
    ax=axes[0],
    title="Reference Image",
    cmap="gray",
)
for idx in range(len(tgt_greys)):
    show(
        rio.open(f"{input_dirs}/tgt_image_{idx}.tif"),
        ax=axes[idx + 1],
        title=f"Target Image {idx}",
        cmap="gray",
    )
plt.tight_layout()

In [ ]:
# some data exploration using geopandas
gdf = gpd.GeoDataFrame.from_features(ref_items + tgt_items)
ids = [item.id for item in ref_items + tgt_items]
gdf["id"] = ids
gdf.plot(
    column="id",
    cmap="viridis",
    alpha=0.7,
    edgecolor="black",
    legend=True,
    figsize=(10, 10),
)

### Reading the file names into input variables

In [ ]:
ref_file = f"{input_dirs}/ref_image.tif"
tgt_files = glob.glob(f"{input_dirs}/tgt_image_*.tif")

In [ ]:
rio.open(ref_file).crs["proj"]

### Reprojecting targets

Reference file has polar stereographic projection (Landsat-8), whereas the target Sentinel-2 files are in UTM coordinates. We must reproject the targets to the reference coordinate system.

In [ ]:
# reproject_tif function from tools.coregister library reprojects a tif file to a specified CRS
ref_epsg = rio.open(
    ref_file
).crs.to_epsg()  # getting the EPSG code of the reference image
for i, tgt in enumerate(tgt_files):
    tgt_epsg = rio.open(tgt).crs.to_epsg()
    print(f"Target image {i} EPSG: {tgt_epsg}")
    if tgt_epsg != ref_epsg:
        reproj_dir = os.path.join(input_dirs, "reprojected")
        os.makedirs(reproj_dir, exist_ok=True)
        print(f"Reprojecting target image {i} to match reference EPSG {ref_epsg}...")
        reprojected_tgt = os.path.join(reproj_dir, os.path.basename(tgt))
        reproject_tif(tgt, reprojected_tgt, rio.open(ref_file).crs)
        tgt_files[i] = reprojected_tgt

## Running Co-Registration tools

we will run `co-register`, `karios` and `arosics` functions from the `tools.coregister` library

### Co-Register

In [ ]:
output_path = f"{output_dirs}/Co_Register"

_, shifts, target_ids = co_register(
    ref_file,
    tgt_files,
    output_path=output_path,
    return_shifted_images=True,
    laplacian_kernel_size=5,
)
print("\nCo-register shifts:")
for i, shift in enumerate(shifts):
    print(
        f"Target {target_ids[i]}: {tuple([np.round(el.tolist(), 3).tolist() for el in shift])} pixels"
    )

### Karios

In [ ]:
output_dir = output_path = f"{output_dirs}/Karios"
shift_dict, target_ids = karios(
    ref_file,
    tgt_files,
    output_dir,
)
print("\nKarios shifts:")
for i, shift in enumerate(shift_dict):
    print(
        f"Target {target_ids[i]}: {tuple([np.round(el, 3).tolist() for el in shift_dict[shift]])} pixels"
    )

### AROSICS

In [ ]:
output_dir = output_path = f"{output_dirs}/Arosics"
shifts, target_ids = arosics(
    ref_file,
    tgt_files,
    output_dir,
)
print("\nAROSICS shifts:")
for i, shift in enumerate(shifts):
    print(
        f"Target {target_ids[i]}: {tuple([np.round(el.tolist(), 3).tolist() for el in shift])} pixels"
    )

### Comparing Results 

Output gifs are generated in the outputs folder under each tool's name. `outputs_raw.gif` is the file generated from the raw inputs before co-registration. `outputs.gif` is the fianl gif generated after co-registration. `output.csv` is the generated csv file for the performance metrics and other reported values.

#### Results table

In [ ]:
output_subdirs = ["Co_Register", "Karios", "Arosics"]
output_pd = pd.DataFrame({"Title": ["target 0", "target 1"]})
for i, output_subdir in enumerate(output_subdirs):
    output_csv = f"{output_dirs}/{output_subdir}/output.csv"
    temp_pd = pd.read_csv(output_csv)
    if i == 0:
        output_pd["SSIM Raw"] = temp_pd["SSIM Raw"]
        output_pd["MSE Raw"] = temp_pd["MSE Raw"]
    output_pd[f"{output_subdir} SSIM"] = temp_pd["SSIM Aligned"]
    output_pd[f"{output_subdir} MSE"] = temp_pd["MSE Aligned"]
    output_pd[f"{output_subdir} Shifts"] = temp_pd["Shifts"]
display(output_pd)

### Results plots

In [ ]:
shifts_pd = output_pd.copy()[
    ["Title"] + [f"{subdir} Shifts" for subdir in output_subdirs]
]
for subdir in output_subdirs:
    shifts_pd[f"{subdir} Shifts x"] = shifts_pd[f"{subdir} Shifts"].apply(
        lambda x: float(x.split(",")[0].replace("(", "").strip())
    )
    shifts_pd[f"{subdir} Shifts y"] = shifts_pd[f"{subdir} Shifts"].apply(
        lambda x: float(x.split(",")[1].replace(")", "").strip())
    )
shifts_pd.drop(columns=[f"{subdir} Shifts" for subdir in output_subdirs], inplace=True)
shifts_pd[["Title"] + [f"{subdir} Shifts x" for subdir in output_subdirs]].plot(
    kind="bar", x="Title", figsize=(10, 6), title="Shifts in x in pixels"
)
shifts_pd[["Title"] + [f"{subdir} Shifts y" for subdir in output_subdirs]].plot(
    kind="bar", x="Title", figsize=(10, 6), title="Shifts in y in pixels"
)

***

## Additional information

**License:** The code in this notebook is licensed under the [Apache License, Version 2.0](https://www.apache.org/licenses/LICENSE-2.0). 
Digital Earth Australia data is licensed under the [Creative Commons by Attribution 4.0](https://creativecommons.org/licenses/by/4.0/) license.

**Contact:** If you need assistance, please post a question on the [Open Data Cube Discord chat](https://discord.com/invite/4hhBQVas5U) or on the [GIS Stack Exchange](https://gis.stackexchange.com/questions/ask?tags=open-data-cube) using the `open-data-cube` tag (you can view previously asked questions [here](https://gis.stackexchange.com/questions/tagged/open-data-cube)).
If you would like to report an issue with this notebook, you can file one on [GitHub](https://github.com/GeoscienceAustralia/dea-notebooks).

**Last modified:** September 2025

## Tags
<!-- Browse all available tags on the DEA User Guide's [Tags Index](https://knowledge.dea.ga.gov.au/genindex/) -->